# 04 – Radial Analysis

Compute radial distance maps from detected islets and summarize cell distributions across radial shells.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from isletscope.segmentation import CellSegmenter
from isletscope.islet_detection import IsletDetector
from isletscope.radial_analysis import RadialAnalyzer

image_path = 'path/to/your/image.tif'
img = cv2.imread(image_path)
if img is None:
    raise FileNotFoundError(f'Cannot load image at {image_path}')

# Segment cells and assume all are endocrine for demonstration
segmenter = CellSegmenter()
seg = segmenter.segment(img)
labels = seg['labels']
endocrine_mask = (labels > 0).astype(np.uint8)

# Detect islets
detector = IsletDetector(min_islet_area=500, min_cell_count=10)
res = detector.detect(endocrine_mask, labels)

# Take first islet for radial analysis
islet_labels = res['islet_labels']
unique_islets = np.unique(islet_labels)
unique_islets = unique_islets[unique_islets > 0]
if len(unique_islets) == 0:
    raise RuntimeError('No islets detected in the image.')
first_islet_id = unique_islets[0]
islet_mask = (islet_labels == first_islet_id).astype(np.uint8)

# Compute distance map and radial bins
analyzer = RadialAnalyzer(bin_size=20)
dist_map, centroid = analyzer.compute_distance_map(islet_mask)
bins = analyzer.radial_bins(dist_map)

# Summarize cell distribution across bins (here all cell pixels)
hist = analyzer.summarize(bins, cell_mask=None)
print('Radial bin counts:', hist)

# Visualize signed distance map
plt.figure(figsize=(5, 4))
plt.imshow(dist_map, cmap='coolwarm')
plt.title('Signed distance map')
plt.colorbar()
plt.axis('off')
plt.show()